In [1]:
include("modle.jl")
using .Model
using StaticArrays
# using Plots
using LinearAlgebra
# using Makie
using GLMakie 

In [2]:
# 定义铜的晶格常数（单位：Å）
lattice_constant = 3.61

# 定义铜的FCC晶胞的基矢量
lattice_vectors = (Matrix([
    lattice_constant 0.0 0.0; #a1
    0.0 lattice_constant 0.0; #a2
    0.0 0.0 lattice_constant] #a3
))'

# 定义铜的FCC晶胞中的原子位置（单位：Å）
atom_positions = [
    Vector([0.0, 0.0, 0.0]),
    Vector([0.0, 0.5, 0.5]),
    Vector([0.5, 0.0, 0.5]),
    Vector([0.5, 0.5, 0.0]),
    Vector([1.0, 0.0, 0.0]),
    Vector([0.0, 1.0, 0.0]),
    Vector([0.0, 0.0, 1.0]),
    Vector([0.5, 1.0, 0.5]),
    Vector([1.0, 0.5, 0.5]),
    Vector([0.5, 0.5, 1.0]),
    Vector([1.0, 0.0, 1.0]),
    Vector([1.0, 1.0, 0.0]),
    Vector([0.0, 1.0, 1.0]),
    Vector([1.0, 1.0, 1.0])
] 

# 创建铜的原子列表
atoms = [Atom(pos) for pos in atom_positions]

cell=UnitCell(lattice_vectors,atoms)
cpcell=copycell(cell,10,10,10)
fig=visualize_unitcell_atoms(cpcell);

In [3]:
#lj势能
function lj(r::Float64)
    return 4*(1/r^12-1/r^6)
end
function Flj(r::Vector{Float64})
    rn=norm(r)
    return -24*(2/rn^14-1/rn^8)*r
end


interaction = Interaction(lj, Flj, 3.0, 1.4)

x=1:0.0001:3
y=interaction.cutenerge.(x)
lines(x,y)

In [4]:
el=Vector{Float64}([])
lattice_constant = 3.61
cl=1.3:0.01:2.5
en=0
for lattice_constant in cl
    # 定义铜的FCC晶胞的基矢量
    lattice_vectors = (Matrix([
        lattice_constant 0.0 0.0; #a1
        0.0 lattice_constant 0.0; #a2
        0.0 0.0 lattice_constant] #a3
    ))
    # 创建铜的原子列表
    atoms = [Atom(pos) for pos in atom_positions]

    cell=UnitCell(lattice_vectors,atoms)
    cpcell=copycell(cell,5,5,5)
    en=cell_energe(cell,interaction,ifnormalize=true)
    push!(el,en)
end


In [5]:
# 创建一个新的 Figure
fig = Figure()

# 创建一个 Axis，并设置坐标范围
ax = Axis(fig[1, 1], title = "Line Plot", xlabel = "X-Axis", ylabel = "Y-Axis",
          )

lines!(ax,cl,el)
# ylims!(ax, -0.001, 0.001)

fig

In [22]:
@code_warntype cell_energe(cell,interaction,ifnormalize=true,tol=1e-3,maxiter=100)

MethodInstance for Core.kwcall(::@NamedTuple{ifnormalize::Bool, tol::Float64, maxiter::Int64}, ::typeof(cell_energe), ::UnitCell, ::Interaction{typeof(lj), typeof(Flj), Main.Model.var"#cutenerge#5"{typeof(lj), Float64, Float64, Float64, Float64, Float64}, Main.Model.var"#cutforce#6"{typeof(Flj), Float64, Float64, Float64, Float64}})
  from kwcall(::NamedTuple, ::typeof(cell_energe), cell::UnitCell, interaction::Interaction) @ Main.Model c:\Users\ASUS\Desktop\计算物理模拟实验\Elastic constant simulation\modle.jl:255
Arguments
  _::Core.Const(Core.kwcall)
  @_2::@NamedTuple{ifnormalize::Bool, tol::Float64, maxiter::Int64}
  @_3::Core.Const(Main.Model.cell_energe)
  cell::UnitCell
  interaction::Interaction{typeof(lj), typeof(Flj), Main.Model.var"#cutenerge#5"{typeof(lj), Float64, Float64, Float64, Float64, Float64}, Main.Model.var"#cutforce#6"{typeof(Flj), Float64, Float64, Float64, Float64}}
Locals
  ifnormalize::Union{}
  maxiter::Union{}
  tol::Union{}
  @_9::Union{Bool, Float64, Int64}
Body:

In [23]:
@code_warntype cell_energeij(cell,interaction,1,2,ifnormalize=true)

MethodInstance for Core.kwcall(::@NamedTuple{ifnormalize::Bool}, ::typeof(cell_energeij), ::UnitCell, ::Interaction{typeof(lj), typeof(Flj), Main.Model.var"#cutenerge#5"{typeof(lj), Float64, Float64, Float64, Float64, Float64}, Main.Model.var"#cutforce#6"{typeof(Flj), Float64, Float64, Float64, Float64}}, ::Int64, ::Int64)
  from kwcall(::NamedTuple, ::typeof(cell_energeij), cell::UnitCell, interaction::Interaction, i::Int64, j::Int64) @ Main.Model c:\Users\ASUS\Desktop\计算物理模拟实验\Elastic constant simulation\modle.jl:192
Arguments
  _::Core.Const(Core.kwcall)
  @_2::@NamedTuple{ifnormalize::Bool}
  @_3::Core.Const(Main.Model.cell_energeij)
  cell::UnitCell
  interaction::Interaction{typeof(lj), typeof(Flj), Main.Model.var"#cutenerge#5"{typeof(lj), Float64, Float64, Float64, Float64, Float64}, Main.Model.var"#cutforce#6"{typeof(Flj), Float64, Float64, Float64, Float64}}
  i::Int64
  j::Int64
Locals
  ifnormalize::Union{}
  maxiter::Union{}
  tol::Union{}
  @_11::Union{Bool, Float64, Int64

In [29]:
@time cell_energe(cpcell,interaction,ifnormalize=true)

 10.870211 seconds (397.09 M allocations: 29.586 GiB, 14.21% gc time, 0.02% compilation time)


-1232.5848183231765